## Experiment-01
### Amirreza Fosoul and Bithiah Yuan

In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import *
import string
import csv
import re
import time
spark = SparkSession.builder.appName('ex1').getOrCreate()

### Exercise 1. 2 (Loading the dataset into an RDD)

In [2]:
userRatingsRDD = spark.sparkContext.textFile('./users_libraries.txt')
paperTermsRDD = spark.sparkContext.textFile('./papers.csv')

In [3]:
# create the pair (userID, [paperIDs])
def makeKeyValue(line):
    line = line.split(';')
    return (line[0], line[1].split(','))

userRatingsPaired = userRatingsRDD.map(lambda line: makeKeyValue(line))

In [4]:
# load the stop words and store them in a list
stopWords = []
with open('./stopwords_en.txt', 'r') as stopWords:
    stopWords = stopWords.readlines()
    stopWords = [word.replace('\n', '') for word in stopWords]

In [39]:
# split the words inside the abstract text of each paper
def splitWords(paper):
    # replace multiple whitespaces with a single one
    paper = re.sub('\s\s+', ' ', paper)
    # split the text by comma(ignore commas inside the quoted strings)
    result = re.split(''',(?=(?:[^'"]|'[^']*'|"[^"]*")*$)''', paper)
    # remove the punctuation signs from the text
    result[-1] = "".join((char for char in result[-1] if char not in string.punctuation))
    # make list of words of the abstract (the last index of the paper detail is the abstract)
    abstract = re.split(' ', result[-1].strip())
    abstract_words = []
    # ignore the words mentioned in the stop words list
    for word in abstract:
        if word not in stopWords:
            abstract_words.append(word)
    # return the pair (paperID, abstractWords)
    return (result[0], abstract_words)

### Exercise 1. 3 (Joining collections)

In [40]:
# filter out the papers without abstracts
paperTermsPaired = paperTermsRDD.map(lambda paper: splitWords(paper)).filter(lambda x: x[1] != [''])

# make the pair ((paperID, word), 1) to start counting
paper_word_count = paperTermsPaired.flatMap(lambda x: [((x[0], word) , 1) for word in x[1]])#.reduceByKey(lambda a, b: a + b)

# reduce based on (paperID, word) to sum up the word frequency of each paper
paper_word_count = paper_word_count.reduceByKey(lambda a, b: a+b)

#convert from ((paperID, word), frequency) to (paperID, (word, frequency))
paper_word_count = paper_word_count.map(lambda x: (x[0][0], (x[0][1], x[1])))

user_paperid = userRatingsPaired.flatMap(lambda x: [(paperID, x[0]) for paperID in x[1]])

# let's join two RDDs to have list of words for each user
joinedRDD = paper_word_count.join(user_paperid) 
# joinedRDD is in the format of (paperID, ((word, counter), userID))
#joinedRDD.collect()

In [41]:
# sort the words for each user and return top 10 words
def top10Words(words):
    result = sorted(words, key=lambda x: x[1], reverse=True)
    if len(result) >= 10:
        return result[:10]
    else:
        # if the user has less than 10 words, return the whole sorted list
        return result

# in this part first we get rid of paperID and then change the format of joinedRDD to ((userID, word), counter)
# then calculate the frquency of each (userID, word) and format it like (userID, (word, counter)). After that group them by the userID and 
# return top 10 words for each user
user_words = joinedRDD.map(lambda x: ((x[1][1], x[1][0][0]), x[1][0][1])).reduceByKey(lambda a, b: a+b).map(lambda x: (x[0][0], (x[0][1], x[1]))).groupByKey().map(lambda x: (x[0], top10Words(list(x[1]))))

start_rdd = time.time()

user_words.collect()

end_rdd = time.time()

runtime_rdd = end_rdd - start_rdd

# Get runtime
print("Execution time for the rdd task is " + str(runtime_rdd))

Execution time for the rdd task is 1274.2255811691284


In [42]:
# Print the ranking of 10 users
user_words.take(10)

[('a88e213aa257eda198f6bd990d3d6fe2',
  [('software', 67),
   ('system', 47),
   ('code', 38),
   ('design', 37),
   ('analysis', 36),
   ('paper', 34),
   ('program', 32),
   ('objectoriented', 28),
   ('approach', 28),
   ('programs', 28)]),
 ('dd70a8664990fd9b300b1596b9fe3963',
  [('information', 110),
   ('users', 79),
   ('systems', 65),
   ('social', 48),
   ('user', 47),
   ('system', 47),
   ('people', 46),
   ('results', 46),
   ('work', 43),
   ('design', 43)]),
 ('06de0a81ec0fe1e9e983961ee02224ad',
  [('social', 56),
   ('design', 46),
   ('research', 29),
   ('paper', 20),
   ('book', 20),
   ('visualization', 19),
   ('work', 18),
   ('data', 17),
   ('information', 17),
   ('community', 15)]),
 ('d57661d5d9e869cf40eb7623fa288592',
  [('tagging', 74),
   ('tags', 57),
   ('users', 55),
   ('social', 49),
   ('systems', 44),
   ('paper', 36),
   ('tag', 35),
   ('collaborative', 28),
   ('web', 28),
   ('user', 22)]),
 ('d5c5cc742fdbcfc356c2aaf45f0ec848',
  [('user', 308),


In [44]:
## Saving rdd result into a file:

## Save rdd result into a file (resulting in a directory with multiple txt files)
#user_words.saveAsTextFile("rdd_output10.txt")

## Read in the result files
#output = spark.sparkContext.textFile('./rdd_output10.txt')
## Output into a single result file
#output.coalesce(1).saveAsTextFile('./rddOutput_final2.txt')

### Exercise 1. 4 (Basic Analysis for Recommender Systems)

In [4]:
# Number of (distinct) user, number of (distinct) items, and number of ratings

start_time = time.time()
num_users = userRatingsRDD.count()
print('Number of users: ' + str(num_users))
print('Execution Time: ' + str(time.time()-start_time) + ' seconds')

start_time = time.time()
num_items = paperTermsRDD.count()
print('\nNumber of items: ' + str(num_items))
print('Execution Time: ' + str(time.time()-start_time) + ' seconds')

start_time = time.time()
num_ratings = userRatingsPaired.map(lambda x: len(x[1])).sum()
print('\nNumber of ratings: ' + str(num_ratings))
print('Execution Time: ' + str(time.time()-start_time) + ' seconds')

#number of ratings for each user, sorted descending

start_time = time.time()
num_ratings_user = userRatingsPaired.map(lambda entry: (entry[0],len(entry[1]))).sortBy(lambda x: x[1])
num_ratings_user_collected = num_ratings_user.collect()

print('\nMax number of ratings a user has given: ' + str(num_ratings_user_collected[::-1][0][1]) + 
      ' by user ' + num_ratings_user_collected[::-1][0][0])
print('Execution Time: ' + str(time.time()-start_time) + ' seconds')

start_time = time.time()
print('\nMin number of ratings a user has given: ' + str(num_ratings_user_collected[0][1]) + 
      ' by user ' + num_ratings_user_collected[0][0])
print('Execution Time: ' + str(time.time()-start_time) + ' seconds')

start_time = time.time()
# average number of ratings of users
print('\nAverage number of ratings of users: ' + str(num_ratings/num_users))
print('Execution Time: ' + str(time.time()-start_time) + ' seconds')

start_time = time.time()
# Standard deviation for ratings of users
print('\nStandard deviation for ratings of users: ' + str(num_ratings_user.map(lambda x: x[1]).stdev()))
print('Execution Time: ' + str(time.time()-start_time) + ' seconds')

# Min/Max number of ratings an item has received
start_time = time.time()
result = userRatingsPaired.flatMap(lambda data: data[1]).map(lambda paper: (paper, 1)).reduceByKey(lambda a, b: a+b).sortBy(lambda x: x[1])
paper_ratings = result.collect()

print('\nMin number of ratings an item has received: ' + str(paper_ratings[0][1]) + ' for the paper ' + str(paper_ratings[0][0]))
print('Max number of ratings an item has received: ' + str(paper_ratings[::-1][0][1]) + ' for the paper ' + str(paper_ratings[::-1][0][0]))
print('Execution Time: ' + str(time.time()-start_time) + ' seconds')

# Average number of ratings of items
start_time = time.time()
total_num_items = result.map(lambda x:x[1]).sum()
print('\nAverage number of ratings of items: ' + str(total_num_items/num_items))
print('Execution Time: ' + str(time.time()-start_time) + ' seconds')

# Standard deviation for ratings of items
start_time = time.time()
print('\nStandard deviation for ratings of items: ' + str(result.map(lambda x: x[1]).stdev()))
print('Execution Time: ' + str(time.time()-start_time) + ' seconds')


Number of users: 28416
Execution Time: 2.6447298526763916 seconds

Number of items: 172079
Execution Time: 4.546404123306274 seconds

Number of ratings: 828481
Execution Time: 0.28208041191101074 seconds

Max number of ratings a user has given: 1922 by user f7d5debb9c7d4d9ab81c63cd26578e23
Execution Time: 1.477747917175293 seconds

Min number of ratings a user has given: 1 by user ca4f1ba4094011d9a8757b1bfcadae5b
Execution Time: 0.0007727146148681641 seconds

Average number of ratings of users: 29.155440596846848
Execution Time: 0.0003135204315185547 seconds

Standard deviation for ratings of users: 81.1751761367
Execution Time: 0.3057866096496582 seconds

Min number of ratings an item has received: 3 for the paper 503574
Max number of ratings an item has received: 924 for the paper 80546
Execution Time: 4.50620174407959 seconds

Average number of ratings of items: 4.81453867119172
Execution Time: 0.4759519100189209 seconds

Standard deviation for ratings of items: 5.47780229231
Execut

### Exercise 1. 5 (Loading the dataset into Data Frames)

In [3]:
############################################### DATAFRAME ########################################################

from pyspark.sql.functions import *
from pyspark.sql.functions import split
import pyspark.sql.functions as f
import re
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, FloatType, StringType
from pyspark.sql.functions import desc

# Read user ratings into Dataframe
user_df = spark.read.option("delimiter", ";").csv('./users_libraries.txt')
user_df = user_df.select(col("_c0").alias("userID"), col("_c1").alias("paperID")) 

# Read in the stopwords as a list
with open('./stopwords_en.txt') as file:
    stopWords = file.read().splitlines()

# Read in records of paper information
w_df = spark.read.csv('./papers.csv')
w_df = w_df.select("_c0", "_c14")
w_df = w_df.select(col("_c0").alias("paperID"), col("_c14").alias("words"))

# Remove punctuations and extra spaces
words_df = w_df.withColumn("words", regexp_replace(col("words"), '[^\w\s]', ""))
words_df = words_df.withColumn("words", regexp_replace(col("words"), '\s\s+', " "))
# Split each word in the abstracts
words_df = words_df.withColumn("words", split("words", " "))
# # Remove papers that has an empty abstract
words_df = words_df.filter((col("words").isNotNull()))
# User defined function to filter stopwords from the column of words
sd = udf(lambda x: [item for item in x if item not in stopWords], ArrayType(StringType()))
words_df = words_df.withColumn("words", sd(col("words")))

# words_df pre-explode (pre-split)
words_df_pre = words_df

### Exercise 1.6 (Tasks on top of DataFrames)

### 1.6.1

In [4]:
# Split (explode) the paperID list per user into individual rows
# Select the paperID w.r.t userID and split the list of paperIDs
# Split the list of paperIDs per user by a comma
# pos is the position of each paper in the array
# val is the paperID
user_df = user_df.select("userID", f.split("paperID", ",").alias("paperID"), f.posexplode(f.split("paperID", ",")).alias("pos", "val"))
user_df = user_df.drop("val")
# Get paperID w.r.t. position in the paperID list
user_df = user_df.select("userID",f.expr("paperID[pos]").alias("paperID"))
user_df = user_df.select("userID", "paperID")

# Count the number of ratings per user
numRatings_user = user_df.groupBy('userID').count()

## Example:
## Print a table with columns of userID and paperID
#user_df.where(user_df.userID=="df5002093c4d5a8448f4002bbb26d43f").show()
## Print a table with userID and the number of ratings w.r.t. the user
#numRatings_user.where(user_df.userID=="df5002093c4d5a8448f4002bbb26d43f").show()

In [5]:
# Split (explode) the list of words into a column of tokens and
# count the number of occurences of the tokens per paper
words_df = words_df.select("paperID", f.explode("words").alias("token")).groupBy("paperID", "token").count()

# Join the users and papers dataframe
df = user_df.join(words_df, ["paperID"])

# For each user merge the tokens and sum up the duplicated tokens
df = df.groupBy("userID", "token").agg(f.collect_list("token").alias("duplicated_values"), f.sum("count").alias("count"))

In [7]:
import time
from pyspark.sql.window import Window
from pyspark.sql.functions import concat, col, lit

df = df.drop("duplicated_values")

# Order the token count in descending order w.r.t. users
window = Window.partitionBy(col("userID")).orderBy((col("count")).desc())

# Create a new column with the row numbers of the count w.r.t. user 
# Get the first 10 rows
df = df.select(col('*'), row_number().over(window).alias('row_number')).where(col('row_number') <= 10)

# Concatenate the word and the count as one column
df = df.withColumn("word_count", f.concat(col("token"), lit(" "), col("count")))
df = df.drop("token", "count")

# Get a dataframe with a column of unique users and a column of the top 10 occurences of words
df = df.groupby("userID").agg(f.concat_ws(", ", f.collect_list(df.word_count)).alias("Rankings"))

# Compute runtime
start = time.time()
df.show()
end = time.time()

runtime = end - start

# Get runtime
print("Execution time for the dataframe task is " + str(runtime))

+--------------------+--------------------+
|              userID|            Rankings|
+--------------------+--------------------+
|03237605301d9dd8e...|genes 31, gene 24...|
|03a537c35019f00cf...|bc 20, mass 12, a...|
|04e7b99118265af25...|learning 23, info...|
|066349c2a8a65717c...|gprd 15, 95 13, c...|
|08f340d9d9ede55ae...|projects 3, boinc...|
|0cf98522336c994a4...|analysis 23, prog...|
|0d875b8672933b9a5...|web 5, 20 3, worl...|
|0de84fbc3c077868e...|health 36, global...|
|125683f9d7eed5af3...|mirnas 63, cancer...|
|13a5067c6e8d3ced6...|health 159, devel...|
|157f60f600cffe9a9...|library 8, librar...|
|164a0015552fbe011...|transporters 7, a...|
|1675f231acf9d1cbb...|nrf2 30, expressi...|
|188b2723f7349804c...|india 3, rushdies...|
|1936cc3b52835fdfc...|neurons 36, stimu...|
|1a37d3c3eaf90d925...|model 2, neuron 2...|
|1c7a5a7c09813054f...|immune 5, model 5...|
|2086aff81a8d6f9d9...|photography 24, w...|
|211103152ac9c903c...|career 30, career...|
|236ef17c11f7f43ac...|agave 16, 

### 1.6.2 (Basic Analysis for Recommender Systems Using Data Frames)

In [8]:
# Join paper and user dataframes to get the count of ratings per item
paper_user = user_df.join(words_df_pre, ["paperID"])
paper_user = paper_user.drop("words")

# Count the number of ratings per paper
numRatings_paper = paper_user.groupBy('paperID').count()

## Example showing the list of users who rated a paper
#paper_user.where(paper_user.paperID=="1103674").show()

## Example showing the count of ratings of a paper
#numRatings_paper.where(numRatings_paper.paperID=="1103674").show()

In [9]:
############################################# DATA ANALYSIS #######################################################
import time
start = time.time()
## NUMBER OF DISTINCT USERS ##

numUsers = user_df.select("userID").distinct().count()
print("\nThe number of distinct users is " + str(numUsers))
end = time.time()
print("Execution time: " + str(end-start))

## NUMBER OF DISTINCT PAPERS ##
start = time.time()
numPapers = w_df.distinct().count()
print("\nThe number of distinct papers is " + str(numPapers))
end = time.time()
print("Execution time: " + str(end-start))

# ## NUMBER OF RATINGS ##
start = time.time()
numRatings = user_df.count()
print("\nThe number of ratings is " + str(numRatings))
end = time.time()
print("Execution time: " + str(end-start))
# ################################### DATA ANALYSIS W.R.T USERS #################################################### 

# ## MIN NUMBER OF RATINGS GIVEN BY A USER ##
start = time.time()
min_num_ratings = numRatings_user.select("count").agg(min("count")).head()[0]
print("\nThe min number of rating(s) given by a user is " + str(min_num_ratings))
end = time.time()
print("Execution time: " + str(end-start))

# ## MAX NUMBER OF RATINGS GIVEN BY A USER ##
start = time.time()
max_num_ratings = numRatings_user.select("count").agg(max("count")).head()[0]
print("\nThe max number of ratings given by a user is " + str(max_num_ratings))
end = time.time()
print("Execution time: " + str(end-start))

# ## AVERAGE NUMBER OF RATINGS OF USERS ## 
start = time.time()
avg_num_ratings = numRatings_user.select("count").agg(mean("count")).head()[0]
avg_num_ratings = float("{0:.2f}".format(avg_num_ratings))
print("\nThe average number of ratings of users is " + str(avg_num_ratings))
end = time.time()
print("Execution time: " + str(end-start))

# ## STANDARD DEVIATION FOR RATINGS OF USERS ## 
start = time.time()
std_ratings = numRatings_user.select("count").agg(stddev("count")).head()[0]
std_ratings = float("{0:.2f}".format(std_ratings))
print("\nThe standard deviation for ratings of users is " + str(std_ratings))
end = time.time()
print("Execution time: " + str(end-start))
# ####################################### DATA ANALYSIS W.R.T PAPERS ################################################

# ## MIN NUMBER OF RATINGS A PAPER HAS RECEIVED ##
start = time.time()
min_paper_ratings = numRatings_paper.select("count").agg(min("count")).head()[0]
print("\nThe min number of rating(s) of a paper is " + str(min_paper_ratings))
end = time.time()
print("Execution time: " + str(end-start))

# ## MAX NUMBER OF RATINGS A PAPER HAS RECEIVED ##
start = time.time()
max_paper_ratings = numRatings_paper.select("count").agg(max("count")).head()[0]
print("\nThe max number of ratings a paper has received is " + str(max_paper_ratings))
end = time.time()
print("Execution time: " + str(end-start))
# ## AVERAGE NUMBER OF RATINGS OF ITEMS

start = time.time()
avg_paper_ratings = numRatings_paper.select("count").agg(mean("count")).head()[0]
avg_paper_ratings = float("{0:.2f}".format(avg_paper_ratings))
print("\nThe average number of ratings of papers is " + str(avg_paper_ratings))
end = time.time()
print("Execution time: " + str(end-start))

# ## STANDARD DEVIATION FOR RATINGS OF ITEMS ##
start = time.time()
std_paper_ratings = numRatings_paper.select("count").agg(stddev("count")).head()[0]
std_paper_ratings = float("{0:.2f}".format(std_paper_ratings))
print("\nThe standard deviation for ratings of papers is " + str(std_paper_ratings))
end = time.time()
print("Execution time: " + str(end-start))


The number of distinct users is 28416
Execution time: 0.883568286895752

The number of distinct papers is 172079
Execution time: 2.7177493572235107

The number of ratings is 828481
Execution time: 0.3659782409667969

The min number of rating(s) given by a user is 1
Execution time: 0.8987569808959961

The max number of ratings given by a user is 1922
Execution time: 1.2685301303863525

The average number of ratings of users is 29.16
Execution time: 1.797194480895996

The standard deviation for ratings of users is 81.18
Execution time: 1.4369292259216309

The min number of rating(s) of a paper is 3
Execution time: 50.070183992385864

The max number of ratings a paper has received is 924
Execution time: 48.16281700134277

The average number of ratings of papers is 5.0
Execution time: 48.622234582901

The standard deviation for ratings of papers is 6.08
Execution time: 48.87841033935547


In [38]:
## Saving the dataframe to file:

## Write df to a parquet directory resulting in multiple parquet files
#df.write.parquet("DF_Final_Results6.parquet")

## Read in parquet directory and convert to a single csv file
#output = spark.read.parquet('DF_Final_Results6.parquet')
#output.toPandas().to_csv("result_empty11.csv", header=False, index=False, sep="\t")

### Analysis:

The execution time for the rdd task was about 20 minutes (1203 s) while for the dataframe task was about 4 minutes (228 s). Using dataframe in Spark is more efficient because the SparkSQL library incorporates performance optimization, therefore the runtime for making queries is faster.

Source (https://databricks.com/blog/2015/04/13/deep-dive-into-spark-sqls-catalyst-optimizer.html)